In [1]:
########################################################################
# File       : CZI_ZARR.ipynb
# Version    : 0.1
# Author     : czsrh
# Date       : 12.11.2019
# Insitution : Carl Zeiss Microscopy GmbH
#
# Disclaimer: Just for testing - Use at your own risk.
# Feedback or Improvements are welcome.
########################################################################

In [2]:

# this can be used to switch on/off warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# import the libraries mentioned above
from apeer_ometiff_library import io, processing, omexmlClass
import czifile as zis
import xmltodict
import os
import numpy as np
import ipywidgets as widgets
import napari
import imgfileutils as imf
import xml.etree.ElementTree as ET
import zarr

In [3]:
# define your testfiles here

testfolder = r'C:\Users\m1srh\Documents\GitHub\ipy_notebooks\Read_OMETIFF_CZI\testdata'

imgdict = {1: os.path.join(testfolder, r'CellDivision_T=10_Z=15_CH=2_DCV_small_green.ome.tiff'),
           2: os.path.join(testfolder, r'CellDivision_T=10_Z=15_CH=2_DCV_small_red.ome.tiff'),
           3: os.path.join(testfolder, r'CellDivision_T=10_Z=15_CH=2_DCV_small.ome.tiff'),
           4: os.path.join(testfolder, r'CellDivision_T=10_Z=15_CH=2_DCV_small_green.czi'),
           5: os.path.join(testfolder, r'CellDivision_T=10_Z=15_CH=2_DCV_small_red.czi'),
           6: os.path.join(testfolder, r'CellDivision_T=10_Z=15_CH=2_DCV_small.czi')
}

filename = imgdict[6]

In [4]:
# read metadata and array differently for OME-TIFF or CZI data
if filename.lower().endswith('.ome.tiff') or filename.lower().endswith('.ome.tif'):
    
    # Return value is an array of order (T, Z, C, X, Y)
    (array, omexml) = io.read_ometiff(filename)
    metadata = imf.get_metadata(filename, series=0)
    
if filename.lower().endswith('.czi'):

    # get the array and the metadata
    array, metadata = imf.get_array_czi(filename)

In [5]:
# outout the shape of the returned numpy array

# shape of numpy array
print('Array Shape: ', array.shape)

# dimension order from metadata
print('Dimension Order (BioFormats) : ', metadata['DimOrder BF Array'])

# shape and dimension entry from CZI file as returned by czifile.py
print('CZI Array Shape : ', metadata['Shape'])
print('CZI Dimension Entry : ', metadata['Axes'])

Array Shape:  (1, 10, 2, 15, 256, 256)
Dimension Order (BioFormats) :  None
CZI Array Shape :  (1, 10, 2, 15, 256, 256, 1)
CZI Dimension Entry :  BTCZYX0


In [19]:
z = zarr.array(array, chunks=(1, 1, 2, 15, 256, 256), dtype='int16')
zarr_info = z.info
print(type(zarr_info))

z.attrs['metadata'] = metadata

print(z.attrs['metadata']['SizeT'])

if type(z) is np.ndarray:
    print('NumPy Array')
elif type(z) is zarr.core.Array:
    print('ZARR')
    

<class 'zarr.util.InfoReporter'>
10
ZARR


In [17]:
# show all the metadata
for key, value in z.attrs['metadata'].items():
    # print all key-value pairs for the dictionary
    print(key, ' : ', value)

Directory  :  C:\Users\m1srh\Documents\GitHub\ipy_notebooks\Read_OMETIFF_CZI\testdata
Filename  :  CellDivision_T=10_Z=15_CH=2_DCV_small.czi
Extension  :  czi
ImageType  :  czi
Name  :  None
AcqDate  :  2016-02-12T09:41:02.4915604Z
TotalSeries  :  None
SizeX  :  256
SizeY  :  256
SizeZ  :  15
SizeC  :  2
SizeT  :  10
Sizes BF  :  None
DimOrder BF  :  None
DimOrder BF Array  :  None
DimOrder CZI  :  {'B': 0, 'S': -1, 'T': 1, 'C': 2, 'Z': 3, 'Y': 4, 'X': 5, '0': 6}
Axes  :  BTCZYX0
Shape  :  (1, 10, 2, 15, 256, 256, 1)
isRGB  :  None
ObjNA  :  1.2
ObjMag  :  50.0
ObjID  :  Objective:1
ObjName  :  Plan-Apochromat 50x/1.2
ObjImmersion  :  Water
XScale  :  0.091
YScale  :  0.091
ZScale  :  0.32
XScaleUnit  :  µm
YScaleUnit  :  µm
ZScaleUnit  :  µm
DetectorModel  :  Axiocam 506
DetectorName  :  Axiocam506m
DetectorID  :  Detector:Axiocam 506
InstrumentID  :  None
Channels  :  ['AF555', 'AF488']
ImageIDs  :  []
PixelType  :  Gray16
SizeM  :  1
SizeB  :  1
SizeS  :  1
SW-Name  :  ZEN 3.1 (blue

In [11]:
# display data using ipy widgets
if metadata['Extension'] == 'ome.tiff':
    ui, out = imf.create_ipyviewer_ome_tiff(array, metadata)
if metadata['Extension'] == 'czi':
    #ui, out = imf.create_ipyviewer_czi(array, metadata)
    ui, out = imf.create_ipyviewer_czi(z, metadata)

display(ui, out)

AttributeError: 'Array' object has no attribute 'min'

In [ ]:
# try to configure napari automatiaclly based on metadata
imf.show_napari(array, metadata)

In [ ]:
# configure napari viewer manually - check array shape and dimensions order carefully 
    
# get the scalefactors
scalefactors = imf.get_scalefactor(metadata)
print(scalefactors)

array = np.squeeze(array, axis=(0, 1))

viewer = napari.Viewer()
# add every channel as a single layer
for ch in range(metadata['SizeC']):
    chname = metadata['Channels'][ch]
    viewer.add_image(array[ch, :, :, :], name=chname, scale=(1, scalefactors['zx'], 1, 1))

jupyter nbconvert MIAP_Read_and_Display_Images.ipynb --to slides --post serve